In [19]:
import os
import pandas as pd

# Paths to the folders containing the mzML files
path = '../Projects/Caitlin_paper/mzml'
# path2 = '../Projects/LIVER_LD_5XFAD/mzml'

# Initialize empty lists to store the file names
file_names = []
# file_names2 = []

# Loop through the files in the first folder
for filename in os.listdir(path):
    # Check if the file has the .mzML extension
    if filename.endswith('.mzML'):
        # Append the filename to the list
        file_names.append(filename)

# Create a pandas DataFrame from the list of file names
df = pd.DataFrame(file_names, columns=['Sample Name'])

# # Loop through the files in the second folder
# for filename in os.listdir(path2):
#     # Check if the file has the .mzML extension
#     if filename.endswith('.mzML'):
#         # Append the filename to the list
#         file_names2.append(filename)

# # Create a second pandas DataFrame from the list of file names in the second folder
# df2 = pd.DataFrame(file_names2, columns=['Sample Name'])

def process_name(name):
    parts = name.split('_')
    # Find the first part that starts with FAD or DOD
    index = next((i for i, part in enumerate(parts) if part.startswith(('FAD', 'DOD'))), None)
    # Return None if no such part exists
    if index is None:
        return None
    # Use all parts starting from the found part and concatenate them
    processed_name = '_'.join(parts[index:])
    # Remove the .mzml suffix if it exists
    return processed_name.replace('.mzml', '')

df['Processed Name'] = df['Sample Name'].apply(process_name)
df2 = pd.DataFrame(df['Processed Name'], columns=['Processed Name'])
df2.head(None)



,Processed Name
0,FAD184_F4_Female_WT_cortex _N1_Brain_Region_8_...
1,FAD184_F4_Female_WT_cerebellum_N1_Brain_Region...
2,DOD94_F3_Female_WT_cerebellum_N1_Brain_Region_...
3,DOD94_F4_Female_5xFAD_cortex _N1_Brain_Region_...
4,FAD185_M3_Male_5xFAD_hippocampus_N1_Brain_Regi...
...,...
277,FAD189_M2_Male_5xFAD_hippocampus_N1_Brain_Regi...
278,FAD189_M1_Male_5xFAD_cerebellum_N1_Brain_Regio...
279,FAD185_M1_Male_5xFAD_hippocampus_N1_Brain_Regi...
280,FAD184_F4_Female_WT_hippocampus_N1_Brain_Regio...


In [20]:
print(df2.columns)


Index(['Processed Name'], dtype='object')


In [29]:
import pandas as pd

# Variables
genotype = ['WT','5xFAD']
brain_region = ['hippocampus','cortex', 'diencephalon', 'cerebellum']
position = 'Not Needed'

# Name transformation function
def transform_name(name):
    # Identify label (e.g., FAD or DOD)
    label = next((part for part in name.split('_') if part.startswith(('FAD', 'DOD'))), None)
    if label is None:
        return [None]*5

    # Extract genotype
    extracted_genotype = next((g for g in genotype if g in name), None)
    if extracted_genotype is None:
        return [None]*5

    # Extract brain region
    extracted_brain_region = next((br for br in brain_region if br in name), None)
    if extracted_brain_region is None:
        return [None]*5

    # Extract sex
    extracted_sex = 'Male' if 'Male' in name else 'Male'
    
    # Extract date
    extracted_date = next((part for part in name.split('_') if part[-8:].isdigit()), "unknown")
    if extracted_date != "unknown":
        extracted_date = extracted_date[-8:]
    
    # Construct the new sample name
    new_name = f"{label}-{extracted_genotype}-{extracted_brain_region}10x_{extracted_date}"
    
    return [new_name, position, extracted_genotype, extracted_brain_region, extracted_sex]

# Apply transformation
df2[['Sample Name', 'Position', 'Genotype', 'Brain Region', 'Sex']] = df2['Processed Name'].apply(transform_name).tolist()

# # Drop rows where 'Sample Name' is None
# df2 = df2.dropna(subset=['Sample Name'])

# Save to CSV
df2.to_csv('transformed_labels.csv', index=False)


AttributeError: 'NoneType' object has no attribute 'split'

In [27]:
df.to_csv('labels_CaitlinPaper.csv', index=False)
# df2.to_csv('labels_liverLDold.csv', index=False)

In [4]:
# Function to parse and extract the key components of a file name
def parse_filename(filename):
    # Split on '__' to separate the date
    parts_before_date, _ = filename.split('__')
    # Replace '-' with '_' to unify the separators, then split
    parts = parts_before_date.replace('-', '_').split('_')
    return parts

# Function to find matches based on defined criteria
def find_matches(row):
    parts1 = parse_filename(row['file_name_x'])
    parts2 = parse_filename(row['file_name_y'])
    # Check for common prefixes
    common_prefixes = ['DG', 'TAG', 'TG']
    if any(prefix in parts1[0] for prefix in common_prefixes) and any(prefix in parts2[0] for prefix in common_prefixes):
        # Check for specific IDs
        specific_ids = ['FAD', 'DOD']
        if any(id in parts1 for id in specific_ids) and any(id in parts2 for id in specific_ids):
            return True
    return False

# # Apply the find_matches function across the Cartesian product of df and df2
# matched_pairs = [(row1['file_name'], row2['file_name']) for _, row1 in df.iterrows() for _, row2 in df2.iterrows() if find_matches(pd.concat([row1, row2]))]

# # Create a DataFrame for matched pairs
# matched_df = pd.DataFrame(matched_pairs, columns=['file_name_1', 'file_name_2'])

matched_pairs = [(row1['file_name'], row2['file_name']) for _, row1 in df.iterrows() for _, row2 in df.iterrows() if row1['file_name'] != row2['file_name'] and find_matches(pd.concat([row1, row2]))]

# Create a DataFrame for matched pairs
matched_df = pd.DataFrame(matched_pairs, columns=['file_name_1', 'file_name_2'])


print(matched_df)


KeyError: 'file_name'